# VGG16

# ResNet50

# Inception

# Xception

In [4]:
#VGG16

In [6]:
import glob
import zipfile
files = glob.glob('./Animal_Dataset.zip')
files
for file in files:
    print('Unzipping:',file)

    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall('animal_dataset')

Unzipping: ./Animal_Dataset.zip


In [9]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np

In [7]:
train_path = './animal_dataset/dataset/Training/'
test_path = './animal_dataset/dataset/Testing/'

In [14]:
train_gen =  ImageDataGenerator(rescale=(1./255),
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True)

test_gen = ImageDataGenerator(rescale=(1./255))

In [15]:
# input_size pf vgg16 is 224 x 224


train = train_gen.flow_from_directory(train_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

test = test_gen.flow_from_directory(test_path,
                                    target_size=(224,224),
                                    batch_size=22,
                                    class_mode='categorical')

Found 1238 images belonging to 4 classes.
Found 326 images belonging to 4 classes.


In [16]:
# in transfer learning, you need to preprocess the input before loading the model

from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16

In [17]:
# adding preprocessing layer in front of VGG   (INPUT LAYER)

vgg = VGG16(include_top=False,               #include_top=False because, we donot need input layers of vgg16 and dataset of the same
            weights='imagenet',
            input_shape=(224,224,3)) 

58889256/58889256 [==============================] - 37s 1us/step


In [18]:
# Train model with existing weights

for layer in vgg.layers:
    print(layer)


In [19]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [20]:
# Train model with existing weights  (pre_trained weights, no new weights)

for layer in vgg.layers:
    layer.trainable=False

In [21]:
X = Flatten()(vgg.output)

In [24]:
# output layer

prediction = Dense(4, activation='softmax')(X)

In [25]:
# create vgg16 model 

model = Model(inputs=vgg.input, outputs=prediction)

In [26]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [29]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
model.fit_generator(train,validation_data=test, epochs=50, steps_per_epoch=len(train), validation_steps=len(test))

C:\Users\Madhan Kumar S\AppData\Local\Temp\ipykernel_24008\415526643.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train,validation_data=test, epochs=50, steps_per_epoch=len(train), validation_steps=len(test))


Epoch 1/50
57/57 [==============================] - 21s 359ms/step - loss: 0.0302 - accuracy: 0.9935 - val_loss: 0.0035 - val_accuracy: 1.0000
Epoch 2/50
57/57 [==============================] - 18s 306ms/step - loss: 0.0177 - accuracy: 0.9976 - val_loss: 0.0080 - val_accuracy: 1.0000
Epoch 3/50
57/57 [==============================] - 19s 322ms/step - loss: 0.0204 - accuracy: 0.9943 - val_loss: 0.0153 - val_accuracy: 1.0000
Epoch 4/50
57/57 [==============================] - 18s 320ms/step - loss: 0.0289 - accuracy: 0.9935 - val_loss: 0.0047 - val_accuracy: 1.0000
Epoch 5/50
57/57 [==============================] - 19s 326ms/step - loss: 0.0198 - accuracy: 0.9935 - val_loss: 0.0072 - val_accuracy: 1.0000
Epoch 6/50
57/57 [==============================] - 19s 330ms/step - loss: 0.0174 - accuracy: 0.9968 - val_loss: 0.0072 - val_accuracy: 0.9969
Epoch 7/50
57/57 [==============================] - 19s 332ms/step - loss: 0.0239 - accuracy: 0.9952 - val_loss: 0.0158 - val_accuracy: 0.9969

In [33]:
model.save('./animal_dataset/animal_vgg.h5')

# RESNET50

In [34]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [36]:
#input layer

resnet = ResNet50(include_top=False,
                  input_shape=(224,224,3))

94765736/94765736 [==============================] - 35s 0us/step


In [37]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [38]:
#use the pre-trained weights

for layer in resnet.layers:
    layer.trainable=False

In [39]:
#flatten layer

X = Flatten()(resnet.output)

In [40]:
# output layer

out = Dense(4,activation='softmax')(X)

In [43]:
#resnet model development

res_model = Model(inputs=resnet.input, outputs=out)

In [44]:
res_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [45]:
res_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [47]:
res_model.fit(train,validation_data=test, epochs=20,steps_per_epoch=len(train),validation_steps=len(test))

Epoch 1/20
57/57 [==============================] - 24s 322ms/step - loss: 2.4269 - accuracy: 0.3837 - val_loss: 1.0629 - val_accuracy: 0.5000
Epoch 2/20
57/57 [==============================] - 16s 276ms/step - loss: 1.2878 - accuracy: 0.5040 - val_loss: 0.9472 - val_accuracy: 0.5767
Epoch 3/20
57/57 [==============================] - 16s 272ms/step - loss: 1.1454 - accuracy: 0.5525 - val_loss: 0.8810 - val_accuracy: 0.6319
Epoch 4/20
57/57 [==============================] - 16s 277ms/step - loss: 1.3440 - accuracy: 0.5452 - val_loss: 0.8288 - val_accuracy: 0.6472
Epoch 5/20
57/57 [==============================] - 16s 272ms/step - loss: 1.3834 - accuracy: 0.5614 - val_loss: 0.7749 - val_accuracy: 0.6748
Epoch 6/20
57/57 [==============================] - 16s 287ms/step - loss: 0.9353 - accuracy: 0.6414 - val_loss: 0.8171 - val_accuracy: 0.6288
Epoch 7/20
57/57 [==============================] - 17s 300ms/step - loss: 1.1550 - accuracy: 0.5646 - val_loss: 0.8276 - val_accuracy: 0.6442

In [48]:
res_model.save('./animal_dataset/animal_resnet50.h5')

# INCEPTION

In [7]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [8]:
#input layer

inception = InceptionV3(include_top=False,
                        input_shape=(299,299,3))

# dataset preprocessing

In [10]:
train_gen =  ImageDataGenerator(rescale=(1./255),
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True)

test_gen = ImageDataGenerator(rescale=(1./255))

In [27]:
# input_size of inception v3 is 299 x 299


train = train_gen.flow_from_directory(train_path,
                                      target_size=(299,299),
                                      batch_size=8,
                                      class_mode='categorical')

test = test_gen.flow_from_directory(test_path,
                                    target_size=(299,299),
                                    batch_size=8,
                                    class_mode='categorical')

Found 1238 images belonging to 4 classes.
Found 326 images belonging to 4 classes.


In [9]:
#use the pre-trained weights

for layer in inception.layers:
    layer.trainable=False

In [10]:
inception.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [11]:
#flatten layer

X = Flatten()(inception.output)

X.shape

TensorShape([None, 131072])

In [12]:
#output layer

out_inc = Dense(4, activation='softmax')(X)


In [13]:
inception_model = Model(inputs = inception.input, outputs = out_inc)

In [14]:
inception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
inception_model.fit(train, validation_data=test, epochs=50, steps_per_epoch=len(train), validation_steps=len(test))



Epoch 1/50
57/57 [==============================] - 60s 816ms/step - loss: 0.8539 - accuracy: 0.9483 - val_loss: 0.0102 - val_accuracy: 0.9969
Epoch 2/50
57/57 [==============================] - 23s 397ms/step - loss: 0.2550 - accuracy: 0.9903 - val_loss: 6.0701e-08 - val_accuracy: 1.0000
Epoch 3/50
57/57 [==============================] - 23s 394ms/step - loss: 0.1734 - accuracy: 0.9919 - val_loss: 0.0283 - val_accuracy: 0.9969
Epoch 4/50
57/57 [==============================] - 24s 413ms/step - loss: 0.2124 - accuracy: 0.9887 - val_loss: 3.6567e-10 - val_accuracy: 1.0000
Epoch 5/50
57/57 [==============================] - 23s 398ms/step - loss: 0.3275 - accuracy: 0.9863 - val_loss: 0.5811 - val_accuracy: 0.9847
Epoch 6/50
57/57 [==============================] - 23s 395ms/step - loss: 0.1200 - accuracy: 0.9952 - val_loss: 0.0835 - val_accuracy: 0.9969
Epoch 7/50
57/57 [==============================] - 23s 395ms/step - loss: 0.3338 - accuracy: 0.9927 - val_loss: 0.1393 - val_accuracy

In [16]:
inception_model.save('./animal_dataset/animal_inceptionV3.h5')

# XCEPTION

In [29]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

RuntimeError: Physical devices cannot be modified after being initialized

In [1]:
import tensorflow as tf

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [5]:
from tensorflow.keras.applications.xception import Xception

In [18]:
#input layer

input_xc = Xception(include_top=False,
                 input_shape=(299,299,3),
                 weights='imagenet')

In [19]:
#use the pretrained weights

for layer in input.layers:
    layer.trainable=False

In [20]:
#Flatten Layer

X = Flatten()(input_xc.output)

In [23]:
#output layer

out_xc = Dense(4, activation='relu')(X)

In [24]:
xception_model = Model(inputs=input_xc.input, outputs=out_xc)

In [25]:
xception_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
xception_model.fit(train,validation_data=test, epochs=50, steps_per_epoch=len(train), validation_steps=len(test))

Epoch 1/50


NotFoundError: Graph execution error:

Detected at node 'model_4/conv2d_7/Conv2D' defined at (most recent call last):
    File "c:\Anaconda\envs\tf_gpu\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Anaconda\envs\tf_gpu\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Madhan Kumar S\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Madhan Kumar S\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Madhan Kumar S\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\Madhan Kumar S\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "